In [ ]:
!python -m pip install --upgrade pip
!pip install -U bitsandbytes pandas transformers peft datasets hf_transfer trl
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 38.0 MB/s  0:00:00 eta 0:00:01
  Preparing metadata (pyproject.toml) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp311-cp311-linux_x86_64.whl size=255974200 sha256=8c44e8ec694edcd9e6eb71316d164a7efa4151e45a2a52f2835f5ef673f405a0
  Stored in directory: /root/.cache/pip/wheels/42/31/1f/4b22dd7295b3cb064b8fa9038f6d58fb15c9571555b2d7c39c
Successfully built flash-attn
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [flash-attn]2 [flash-attn]


In [1]:
import os
import json
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset, Dataset

from make_prompts import generate_prompts
from qlora import load_model_and_tokenizer, load_sft_model_and_tokenizer

with open("config.json", "r") as f:
    cfg = json.load(f)

#### Load Dataset

In [2]:
dataset = load_dataset(cfg["dataset"], split="train")

In [3]:
# 문자열 최대 길이 제한 해제
pd.set_option("display.max_colwidth", None)

# 컬럼/행 개수도 충분히 넉넉하게
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

df = dataset.to_pandas()
print(df[['query', 'test', 'response']].head(1))

#### baseline 확인

In [4]:
# Qwen2.5 프롬프트 형식 확인
MODEL_NAME = cfg["model_name"]

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

...

Some parameters are on the meta device because they were offloaded to the disk.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [14]:
print(tokenizer.special_tokens_map)

{'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}


In [1]:
query = df['query'][1972]
response = df['response'][1972]

system_msg, user_msg = query.split("### Question:", 1)

pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=2048)

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": user_msg},
    # {"role": "assistant", "content": response}
]

prompt = pipe.tokenizer.apply_chat_template(messages, 
                                            tokenize=False, 
                                            add_generation_prompt=True)


print(prompt)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/masang/miniconda/envs/langchain/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/qj/sfy11gcx28d6_5n6tf2m8wc00000gn/T/ipykernel_78909/4063063970.py", line 1, in <module>
    query = df['query'][1972]
            ^^
NameError: name 'df' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/masang/miniconda/envs/langchain/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/masang/miniconda/envs/langchain/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users

In [18]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=1.0,
    top_p=0.95,
    add_special_tokens=True
)

print(outputs[0]["generated_text"][len(prompt):])

: 

#### SFT

In [6]:
MODEL_NAME = cfg["model_name"]
model, tokenizer = load_model_and_tokenizer(MODEL_NAME)
texts = generate_prompts(df, tokenizer)
train_ds = Dataset.from_dict({"text": texts})
output_dir = "checkpoints"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
sft_config = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=cfg["sft"]["num_epochs"],
    per_device_train_batch_size=cfg["sft"]["batch_size"],
    save_strategy="epoch",
    optim=cfg["sft"]["optim"],
    learning_rate=cfg["sft"]["lr"],
    bf16=True,
    completion_only_loss=True,
    dataset_text_field="text",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    args=sft_config
)

Adding EOS to train dataset:   0%|          | 0/2641 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2641 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2641 [00:00<?, ? examples/s]

In [ ]:
trainer.train()
trainer.save_model(output_dir)

model_path = os.path.join(output_dir, cfg["ckpt_name"])
trainer.save_model(model_path)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Casting fp32 inputs back to torch.bfloat16 for flash-attn compatibility.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,1.001600
20,0.715400
30,0.572200
40,0.477500
50,0.387600
60,0.330100
70,0.311000
80,0.299000
90,0.273600
100,0.289200


In [31]:
sft_model, sft_tokenizer = load_sft_model_and_tokenizer(model_path)

NameError: name 'load_sft_model_and_tokenizer' is not defined

In [11]:
test_dataset = load_dataset(cfg["dataset"], split="test")

In [22]:
test_df = test_dataset.to_pandas()
test_idx = 100

query = test_df['query'][test_idx]
system_msg, question = query.split("### Question:", 1)

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": question}
]

prompt = tokenizer.apply_chat_template(messages, 
                                       tokenize=False, 
                                       add_generation_prompt=True)

test_pipeline = pipeline("text-generation", 
                         model=sft_model, 
                         tokenizer=tokenizer, 
                         max_new_tokens=cfg["generation"]["max_new_tokens"])

# 생성
outputs = test_pipeline(
    prompt,
    do_sample=True,
    temperature=cfg["generation"]["temperature"],
    top_p=cfg["generation"]["top_p"],
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

Device set to use cuda:0


To solve this problem, we need to efficiently process the range updates in `nums` and determine the smallest value \( k \) such that after applying the first \( k \) queries, `nums` becomes a Zero Array. The approach involves using a Binary Indexed Tree (BIT) to keep track of the number of values in the range `[li, ri]` that need to be decremented. We then simulate the effects of the queries on `nums` until it becomes a Zero Array.

Here's a step-by-step implementation plan:

1. **Initialize the BIT**:
   - Create a BIT to keep track of the sum of values in ranges of `nums`.

2. **Process each query**:
   - For each query, update the BIT to decrement the values in the specified range `[li, ri]` by `vali`.
   - Simulate the effect of these range updates on `nums` to determine if it becomes a Zero Array.

3. **Check for Zero Array**:
   - After processing the first \( k \) queries, check if `nums` becomes a Zero Array.
   - If it does, return \( k \). Otherwise, continue processing more 

In [23]:
print("question:", test_df.iloc[test_idx]["query"])
print("response:", test_df.iloc[test_idx]["response"])

question: You are an expert Python programmer. You will be given a question (problem specification) and will generate a correct Python program that matches the specification and passes all tests.

### Question:
You are given an integer array nums of length n and a 2D array queries where queries[i] = [li, ri, vali].
Each queries[i] represents the following action on nums:

Decrement the value at each index in the range [li, ri] in nums by at most vali.
The amount by which each value is decremented can be chosen independently for each index.

A Zero Array is an array with all its elements equal to 0.
Return the minimum possible non-negative value of k, such that after processing the first k queries in sequence, nums becomes a Zero Array. If no such k exists, return -1.
 
Example 1:

Input: nums = [2,0,2], queries = [[0,2,1],[0,2,1],[1,1,3]]
Output: 2
Explanation:

For i = 0 (l = 0, r = 2, val = 1):

Decrement values at indices [0, 1, 2] by [1, 0, 1] respectively.
The array will become [1